In [ ]:
from google.colab import drive
drive.mount("/content/drive")
#drive.mount("/content/drive", force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
% cd 'drive/My Drive/219 Project 4'
% ls

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-29-0113ce680fc7>", line 1, in <module>
    get_ipython().magic("cd 'drive/My Drive/219 Project 4'")
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2160, in magic
    return self.run_line_magic(magic_name, magic_arg_s)
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2081, in run_line_magic
    result = fn(*args,**kwargs)
  File "<decorator-gen-84>", line 2, in cd
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/magic.py", line 188, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/magics/osm.py", line 288, in cd
    oldcwd = py3compat.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

During handling of the 

OSError: ignored

In [ ]:
from IPython.display import display
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import SCORERS
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import PolynomialFeatures
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def convert_categorical(df, cols):
    for c in cols:
        df[c] = pd.Categorical(df[c])
    return df

In [ ]:
def enc_df(df, cat):
    X = df.copy(deep = True)
    enc = OrdinalEncoder()
    enc.fit(X[cat])
    X[cat] = enc.transform(X[cat])
    
    #enc is returned so as to transform any test data, if needed.
    return X, enc

In [ ]:
df_bik = pd.read_csv('df_bik.csv')

df_sui = pd.read_csv('df_sui.csv',keep_default_na=False)

df_vid = pd.read_csv('df_vid.csv')
df_vid = df_vid.drop(columns=['umem'])

In [ ]:
y_bik = 'cnt'
y_sui = 'suicides/100k pop'
y_vid = 'utime'
ys = [y_bik, y_sui, y_vid]

dfs = [df_bik, df_sui, df_vid]

datasets = ['Bike Sharing', 'Suicide', 'Video Transcoding']

In [ ]:
cat_bik = ['season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit']
cat_bik_mask = np.array([1,1,1,1,1,1,1,0,0,0,0])

cat_sui = ['year', 'sex', 'age', 'generation','continent']
cat_sui_mask = np.array([1,1,1,0,0,0,1,1])

cat_vid = ['codec', 'o_codec']
cat_vid_mask = np.array([0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0])

cat_masks = [cat_bik_mask, cat_sui_mask, cat_vid_mask]
cat_names = [cat_bik, cat_sui, cat_vid]

In [ ]:
df_bik = convert_categorical(df_bik, cat_bik)
df_sui = convert_categorical(df_sui, cat_sui)
df_vid = convert_categorical(df_vid, cat_vid)

dfs = [df_bik, df_sui, df_vid]

In [ ]:
df_bik_oh = pd.get_dummies(df_bik)
df_sui_oh = pd.get_dummies(df_sui)
df_vid_oh = pd.get_dummies(df_vid)

dfs_oh = [df_bik_oh, df_sui_oh, df_vid_oh]

In [ ]:
df_bikenc, enc_bik = enc_df(df_bik, cat_bik)
df_suienc, enc_sui = enc_df(df_sui, cat_sui)
df_videnc, enc_vid = enc_df(df_vid, cat_vid)

dfs_enc = [df_bikenc, df_suienc, df_videnc]

In [ ]:
#Checking for any null values
print(df_bik.isnull().sum().sum())
print(df_sui.isnull().sum().sum())
print(df_vid.isnull().sum().sum())

0
0
0


In [ ]:
def Poly_Reg_twoal(d, df, Y, drp,alphas,deg,title):
    X = df.drop(columns = [Y] + drp)
    y = df[Y]
    X = pd.get_dummies(X)

    cols = X.columns
    print("No of features", len(cols))
    feat_names = ['x'+str(i) for i in range(len(cols))]
    dfpoly = pd.DataFrame({'col_name':cols, 'col_rep':feat_names})
    display(dfpoly)

    poly = PolynomialFeatures(deg)
    X = poly.fit_transform(X)

    alphar, alphal = alphas

    rid_scores={}
    las_scores={}
    
    rid_train_rmse = []
    rid_test_rmse = []
    las_train_rmse = []
    las_test_rmse = [] 

    rid_reg = Ridge(alpha = alphar)
    scores = cross_validate(rid_reg, X, y, scoring="neg_mean_squared_error", cv=10, return_train_score=True)
    rid_train_rmse.append(np.sqrt(np.mean(-scores['train_score'])))
    rid_test_rmse.append(np.sqrt(np.mean(-scores['test_score'])))
    
    las_reg = Lasso(alpha = alphal)
    scores = cross_validate(las_reg, X, y, scoring="neg_mean_squared_error", cv=10, return_train_score=True)
    las_train_rmse.append(np.sqrt(np.mean(-scores['train_score'])))
    las_test_rmse.append(np.sqrt(np.mean(-scores['test_score'])))
    
  
    dframe = pd.DataFrame({'Ridge Train RMSE':rid_train_rmse, 'Ridge Valid RMSE':rid_test_rmse})
    print(d+" Ridge Rigression deg {} ".format(deg)+title)
    display(dframe)
    
    dframe = pd.DataFrame({'Lasso Train RMSE':las_train_rmse, 'Lasso Valid RMSE':las_test_rmse})
    print(d+" Lasso Rigression deg {} ".format(deg)+title)
    display(dframe)

Invert Frame rate and see regression performance

In [ ]:
inv_frate = (df_vid['framerate'])**-1
X = df_vid.drop(columns = 'framerate')
X['inv_fr'] = inv_frate

In [ ]:
deg = 2
Poly_Reg_twoal(datasets[2],X, ys[2],drp = [],alphas = (1000,0.1), deg = deg, title = "")

No of features 24


,col_name,col_rep
0,duration,x0
1,width,x1
2,height,x2
3,bitrate,x3
4,i,x4
5,p,x5
6,b,x6
7,frames,x7
8,i_size,x8
9,p_size,x9


Video Transcoding Ridge Rigression deg 2 


,Ridge Train RMSE,Ridge Valid RMSE
0,6.333415,14.185797


Video Transcoding Lasso Rigression deg 2 


,Lasso Train RMSE,Lasso Valid RMSE
0,6.584652,7.288184


Invert number of frames:

In [ ]:
inv_fr = (df_vid['frames'])**-1.0
X = df_vid.drop(columns = ['frames'])
X['inv_fr'] = inv_fr
deg = 2
Poly_Reg_twoal(datasets[2],X, ys[2],drp = [],alphas = (1000,0.1), deg = deg, title = "")

No of features 24


,col_name,col_rep
0,duration,x0
1,width,x1
2,height,x2
3,bitrate,x3
4,framerate,x4
5,i,x5
6,p,x6
7,b,x7
8,i_size,x8
9,p_size,x9


Video Transcoding Ridge Rigression deg 2 


,Ridge Train RMSE,Ridge Valid RMSE
0,6.322121,21.227671


Video Transcoding Lasso Rigression deg 2 


,Lasso Train RMSE,Lasso Valid RMSE
0,6.563849,7.484193


In [ ]:
inv_fr = (df_vid['frames'])**-1.0
inv_frate = (df_vid['framerate'])**-1.0
X = df_vid.drop(columns = ['frames', 'framerate'])
X['inv_fr'] = inv_fr
X['inv_frate'] = inv_frate
deg = 2
Poly_Reg_twoal(datasets[2],X, ys[2],drp = [],alphas = (1000,0.1), deg = deg, title = "")

No of features 24


,col_name,col_rep
0,duration,x0
1,width,x1
2,height,x2
3,bitrate,x3
4,i,x4
5,p,x5
6,b,x6
7,i_size,x7
8,p_size,x8
9,size,x9


Video Transcoding Ridge Rigression deg 2 


,Ridge Train RMSE,Ridge Valid RMSE
0,6.32069,8.311974


Video Transcoding Lasso Rigression deg 2 


,Lasso Train RMSE,Lasso Valid RMSE
0,6.569238,7.134968


In [ ]:
def mlp(d,df,Y,hid_lay_size,alpha,lr,batch_size):

    X = df.drop(columns = Y)
    X = pd.get_dummies(X)
    y = df[Y]
    
    nn_reg = MLPRegressor(hid_lay_size, activation = 'relu', alpha = alpha, max_iter=300,learning_rate_init= lr,batch_size = batch_size, learning_rate = 'adaptive', early_stopping = True)
    scores = cross_validate(nn_reg, X, y, scoring="neg_mean_squared_error", cv=10, return_train_score=True)
          
    return np.sqrt(np.mean(-scores['test_score'])), np.sqrt(np.mean(-scores['train_score']))

In [ ]:
#hid_size_list = [[200,500,500,200],[300,600,600,600,200],[500,1000,1000,500]]
#al_list = [1e-1,1e1,1e2,1e3]
#lr_list = [1e-4, 5e-3, 1e-1]
hid_size_list = [[300,600,600,600,200]]
al_list = [1e2]
lr_list = [1e-3]
d = datasets[1]
df = dfs[1]
Y = ys[1]

nn_list = []
print(d+" MLP")
for hid_size in hid_size_list:
    for al in al_list:
        for lr in lr_list:
            val_score, tr_score = mlp(d,df,Y,hid_size, al,lr,1024)
            nn_list.append([hid_size, val_score, tr_score, lr,al])
nn_df = pd.DataFrame(nn_list, columns = ['Hidden Size', 'Avg Valid RMSE', 'Avg Train RMSE', 'LR','Alpha'])
display(nn_df)

Suicide MLP


,Hidden Size,Avg Valid RMSE,Avg Train RMSE,LR,Alpha
0,"[300, 600, 600, 600, 200]",14.768253,12.141768,0.001,100.0


In [ ]:
hid_size_list = [[300, 500, 200],[200,500,500,200],[300,600,600,600,200]]
al_list = [1e-1,1e1,1e2,1e3]
lr_list = [1e-4, 5e-3, 1e-1]
for d, df, Y in zip(datasets, dfs, ys):
    nn_list = []
    print(d+" MLP")
    for hid_size in hid_size_list:
        for al in al_list:
            for lr in lr_list:
                val_score, tr_score = mlp(d,df,Y,hid_size, al,lr)
                nn_list.append([hid_size, val_score, tr_score, lr])
    nn_df = pd.DataFrame(nn_list, columns = ['Hidden Size', 'Avg Valid RMSE', 'Avg Train RMSE', 'LR'])
    display(nn_df)

Bike Sharing MLP


,Hidden Size,Avg Valid RMSE,Avg Train RMSE,LR
0,"[300, 500, 200]",945.991572,771.994056,0.0001
1,"[300, 500, 200]",913.062202,658.613518,0.0050
2,"[300, 500, 200]",1236.097119,1201.043316,0.1000
3,"[300, 500, 200]",1066.346358,856.686661,0.0001
4,"[300, 500, 200]",862.294441,700.119869,0.0050
5,"[300, 500, 200]",901.896137,754.942178,0.1000
6,"[300, 500, 200]",910.431761,765.503430,0.0001
7,"[300, 500, 200]",851.401551,653.799133,0.0050
8,"[300, 500, 200]",836.028609,501.176772,0.1000
9,"[300, 500, 200]",890.704954,757.110922,0.0001


Suicide MLP


In [ ]:
def mlp_noCV(d,df,Y,hid_lay_size,alpha,batch_size,lr):

    X = df.drop(columns = Y)
    X = pd.get_dummies(X)
    y = df[Y]
    X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1)
    nn_reg = MLPRegressor(hid_lay_size, activation = 'relu', alpha = alpha, max_iter=300,learning_rate_init= lr,batch_size = batch_size, learning_rate = 'adaptive', early_stopping = True)
    nn_reg.fit(X_train, y_train)
    train_pred = nn_reg.predict(X_train)
    test_pred = nn_reg.predict(X_test)

    return mean_squared_error(y_train,train_pred), mean_squared_error(y_test,test_pred)

In [ ]:
hid_size_list = [[300,600,600,600,200]]
al_list = [1e3]
lr_list = [1e-3]
d = datasets[1]
df = dfs[1]
Y = ys[1]

nn_list = []
print(d+" MLP")
for hid_size in hid_size_list:
    for al in al_list:
        for lr in lr_list:
            tr_score, val_score = mlp_noCV(d,df,Y,hid_size, al,512,lr)
            nn_list.append([hid_size, val_score, tr_score, lr,al])
nn_df = pd.DataFrame(nn_list, columns = ['Hidden Size', 'Avg Valid RMSE', 'Avg Train RMSE', 'LR','Alpha'])
display(nn_df)

Suicide MLP


,Hidden Size,Avg Valid RMSE,Avg Train RMSE,LR,Alpha
0,"[300, 600, 600, 600, 200]",166.740852,167.730007,0.001,1000.0


In [ ]:
hid_size_list = [[200,500,500,200],[300,600,600,600,200]]
al_list = [1e-2,1e1,1e2,1e3]
lr_list = [ 5e-3]
d = datasets[1]
df = dfs[1]
Y = ys[1]

nn_list = []
print(d+" MLP")
for hid_size in hid_size_list:
    for al in al_list:
        for lr in lr_list:
            tr_score, val_score = mlp_noCV(d,df,Y,hid_size, al,512,lr)
            nn_list.append([hid_size, val_score, tr_score, lr,al])
nn_df = pd.DataFrame(nn_list, columns = ['Hidden Size', 'Avg Valid RMSE', 'Avg Train RMSE', 'LR','Alpha'])
display(nn_df)

Suicide MLP


,Hidden Size,Avg Valid RMSE,Avg Train RMSE,LR,Alpha
0,"[200, 500, 500, 200]",158.709368,109.620041,0.005,0.01
1,"[200, 500, 500, 200]",154.103722,106.100199,0.005,10.00
2,"[200, 500, 500, 200]",142.941868,104.676267,0.005,100.00
3,"[200, 500, 500, 200]",180.350828,184.202084,0.005,1000.00
4,"[300, 600, 600, 600, 200]",146.117978,67.098693,0.005,0.01
5,"[300, 600, 600, 600, 200]",135.497625,79.403298,0.005,10.00
6,"[300, 600, 600, 600, 200]",140.504993,110.779732,0.005,100.00
7,"[300, 600, 600, 600, 200]",171.475260,173.471465,0.005,1000.00


In [ ]:
df_vid.columns

Index(['duration', 'codec', 'width', 'height', 'bitrate', 'framerate', 'i',
       'p', 'b', 'frames', 'i_size', 'p_size', 'size', 'o_codec', 'o_bitrate',
       'o_framerate', 'o_width', 'o_height', 'utime'],
      dtype='object')